In [ ]:
import os
import sys
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class StderrFilter:
    def __init__(self, original_stderr):
        self.original_stderr = original_stderr

    def write(self, message):
        if "FaceBlendshapesGraph" in message: return
        if "Feedback manager requires" in message: return
        if "TensorFlow Lite XNNPACK" in message: return
        if message.strip() == "": return
        self.original_stderr.write(message)

    def flush(self):
        self.original_stderr.flush()

sys.stderr = StderrFilter(sys.stderr)
warnings.filterwarnings('ignore')

sys.path.append('..')
import module

In [ ]:
from src.apex.modules import (
    ApexWindowDataSource,
    ApexFullPhaseDataSource,
    ApexHybridDataSource,
)

In [ ]:
EXTRACTION_MODE = "roi"
WINDOW_K = 5
TARGET_LENGTH = 32
MAX_SUBJECTS = None

In [ ]:
window_datasource = ApexWindowDataSource(k=WINDOW_K, 
                                         mode=EXTRACTION_MODE, 
                                         max_subjects=MAX_SUBJECTS)

full_phase_datasource = ApexFullPhaseDataSource(target_length=TARGET_LENGTH, 
                                                mode=EXTRACTION_MODE, 
                                                max_subjects=MAX_SUBJECTS)

hybrid_datasource = ApexHybridDataSource(target_length=TARGET_LENGTH, 
                                         mode=EXTRACTION_MODE, 
                                         max_subjects=MAX_SUBJECTS)

In [ ]:
def run_preextraction(datasource, strategy_label):
    """Iterasi seluruh datasource untuk memastikan cache terbentuk."""
    total_samples = len(datasource)
    failed_indices = []

    for current_index in range(total_samples):
        try:
            _ = datasource[current_index]
        except Exception:
            failed_indices.append(current_index)

    extracted_count = total_samples - len(failed_indices)
    print(f"{strategy_label}: {extracted_count}/{total_samples} cached")

Pre-extraction untuk setiap strategi dijalankan secara terpisah per cell.

In [ ]:
run_preextraction(window_datasource, "window")

In [ ]:
run_preextraction(full_phase_datasource, "full_phase")

In [ ]:
run_preextraction(hybrid_datasource, "hybrid")